# MNIST 손글씨 - CNN(Convolutional Neural Network) 1
- 2 Conv2d layers
- 1 Fully Connected Network layer
- 1 Output layer
- CNN: 데이터의 특징을 추출하여 특징들의 패턴을 파악하는 구조

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# 0 ~ 1 사이의 값으로 정규화 
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
# 3차원 데이터를 4차원으로 (데이터는 2차원을 3차원으로)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)


## 모델 정의, 설정. 학습, 평가

In [6]:
# Flatten(2차원을 1차원으로)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [8]:
# 32, (3,3): 마스크의 갯수, 커널사이즈
model = Sequential([
    Conv2D(32, (3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),       # 여기까지 2차원
    Flatten(),                                  # 1차원으로 변환
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')             # output의 노드가 10개
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               18874880  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [10]:
# 모델 최적화를 위한 설정
modelpath  = 'model/best_mnist.cnn_1.h5'
checkpoint = ModelCheckpoint(
    modelpath, verbose=1, save_best_only=True
)
early_stopping = EarlyStopping(patience=10)

In [11]:
# 모델 학습
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=50, batch_size=200, verbose=0,
                    callbacks=[checkpoint, early_stopping]
                    )


Epoch 00001: val_loss improved from inf to 0.05080, saving model to model\best_mnist.cnn_1.h5

Epoch 00002: val_loss did not improve from 0.05080

Epoch 00003: val_loss improved from 0.05080 to 0.04662, saving model to model\best_mnist.cnn_1.h5

Epoch 00004: val_loss did not improve from 0.04662

Epoch 00005: val_loss did not improve from 0.04662

Epoch 00006: val_loss did not improve from 0.04662

Epoch 00007: val_loss did not improve from 0.04662

Epoch 00008: val_loss did not improve from 0.04662

Epoch 00009: val_loss did not improve from 0.04662

Epoch 00010: val_loss did not improve from 0.04662

Epoch 00011: val_loss did not improve from 0.04662

Epoch 00012: val_loss did not improve from 0.04662

Epoch 00013: val_loss did not improve from 0.04662


In [14]:
from tensorflow.keras.models import load_model
best_model = load_model(modelpath)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.0348 - accuracy: 0.9880


[0.03477019444108009, 0.9879999756813049]